# Support Vector Machine

In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#import all the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn import preprocessing
import urllib

# To ignore some warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
#loading in building-level file & probabilities

url = 'https://raw.githubusercontent.com/wshenyc/MLForCities/main/Data/building_data_final.csv'
df = pd.read_csv(url)

url_2 = 'https://raw.githubusercontent.com/wshenyc/MLForCities/main/Data/pred_prob_dt_rf_final.csv'
df_prob = pd.read_csv(url_2)

In [3]:
X = df.loc[:, "bbl":"building_class_Z9"]
Y = df.loc[:, 'outcome']

In [4]:
# Get column names first
names = X.columns
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(X)
scaled_df = pd.DataFrame(scaled_df, columns=names) 

X = scaled_df

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=99)

from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV 

svm = LinearSVC(dual=False)
clf = CalibratedClassifierCV(svm) 
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_train)
score = clf.score(X_train, Y_train)
print("Score",score)


Score 0.8957096892897918


In [6]:
correct=1.0*(clf.predict(X_test)==np.asarray(Y_test)).sum()/len(Y_test)
print("Out of sample, linear model successfully predicts {} percent of the data".format((correct)*100))

Out of sample, linear model successfully predicts 89.45442703344699 percent of the data


In [7]:
df_prob_svm = pd.concat([df_prob, 
                 pd.DataFrame(clf.predict_proba(X.loc[:, "bbl":"building_class_Z9"])[:,1],
                              columns = ["prob_vio_svm"])], axis = 1)

df_prob_svm

,bbl,prob_vio_dt,prob_vio_rf,prob_vio_svm
0,3047940001,0.060268,0.171337,0.087187
1,3047940017,0.001839,0.009545,0.077797
2,3047940078,0.001839,0.010462,0.100191
3,2032790058,0.157187,0.181260,0.108772
4,1011980126,0.011708,0.057772,0.091202
...,...,...,...,...
158180,1011980055,0.001839,0.010103,0.076984
158181,2039340058,0.001839,0.012765,0.092161
158182,3087140010,0.947761,0.741642,0.183677
158183,2027860012,0.800000,0.413464,0.187388


In [8]:
df_prob_svm.to_csv("pred_prob_dt_rf_svm.csv", index=False)

In [9]:
def print_confusion_matrix(predictions, labels):
    # Function for printing Confusion matrix, based on labels and predictions
    ls = list(confusion_matrix(predictions, labels)) ## returns tn, fp, fn, tp
    ls = pd.DataFrame(ls, columns = ['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    print ("Confusion Matrix:\n", ls)

In [10]:

print_confusion_matrix(clf.predict(X_test), Y_test)

Confusion Matrix:
                  Predicted Negative  Predicted Positive
Actual Negative               45560                5470
Actual Positive                  35                1137
